# NOAA Coastal LiDAR

This notebook illustrates search, directly loading DEMs, and downloading DEMs and LPCs for local processing

| Dataset                      | Alias      | Type      | Start      | End        | Extent         | Source                                                                      |
| ---------------------------- | ---------- | --------- | ---------- | ---------- | -------------- | --------------------------------------------------------------------------- |
| NOAA Coastal LiDAR           | noaa       | LiDAR     | 1996-10-09 |            | US Territories | [NOAA](https://coast.noaa.gov/digitalcoast/data/coastallidar.html)          |

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box

## Search 

We'll search for data in Florida

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/FL/shape.geojson"
)
m = aoi.explore(color="black")

gf_noaa = coincident.search.search(
    dataset="noaa", intersects=aoi, datetime=["2022-10-27"]
)
print(f"Found {len(gf_noaa)} NOAA scenes")
gf_noaa

In [ ]:
# Plot the entire footprint for this scene and a subset region
buffer_size = 0.02
centroid = gf_noaa.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)
m = gf_noaa.explore(color="black")
mini_aoi.explore(m=m, color="magenta")

## Get Data

```{note}
Our coincident.search.search(dataset="noaa") returns the dataset ids "Lidar Datasets at NOAA Digital Coast" whereas coincident.io.xarray.load_noaa_dem() requires the ids from the "Imagery and Elevation Raster Datasets at NOAA Digital Coast" dataset. The corresponding elevation raster dataset id is the same as the lidar dataset id + 1. 
e.g. "Great Bay NERR UAS Lidar" has id 10175 for lidar data and id 10176 for dem data
```

In [ ]:
noaa_dem_id = int(gf_noaa.id.item()) + 1
print(f"NOAA LiDAR id: {gf_noaa.id.item()}  NOAA DEM id: {noaa_dem_id}")

### Stream 

First we'll show how to stream gridded DEMs directly into Xarray

```{warning}
The larger the NOAA flight, the longer the below function takes regardless of your input AOI
```


In [ ]:
%%time
da_noaa_dem = coincident.io.xarray.load_noaa_dem(mini_aoi, noaa_dem_id)

In [ ]:
da_noaa_dem

In [ ]:
da_noaa_dem.coarsen(x=50, y=50, boundary="trim").mean().plot.imshow();

### Download 

```{note}
You will not see a download progress bar for downloading NOAA DEMs as opposed to USGS and NEON provider DEMs
```

#### DEM

In [ ]:
%%time
local_output_dir = "/tmp"

coincident.io.download.download_noaa_dem(
    aoi=mini_aoi, dataset_id=noaa_dem_id, output_dir=local_output_dir
)

In [ ]:
!ls -tlrh /tmp/clipped*tif

#### LPC Tiles

In [ ]:
%%time

# Even smaller AOI
buffer_size = 0.008
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)

gf_noaa_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=mini_aoi, dataset_id=gf_noaa.id.item(), provider="NOAA"
)

In [ ]:
gf_noaa_lpc_tiles

In [ ]:
m = mini_aoi.explore(color="black")
gf_noaa_lpc_tiles.explore(m=m, color="magenta")